In [4]:
from pynq import DefaultIP
from pynq import Overlay

import random 
import numpy as np
import time

In [45]:
class matrix_mult_driver:
    def __init__(self, path):
        ##Creating the overlay, and initializing the IPs
        self.overlay = Overlay(path)
        
        ##BRAM Initializing
        self.bram1 = self.overlay.axi_bram_ctrl_0
        self.bram2 = self.overlay.axi_bram_ctrl_1
        self.bram3 = self.overlay.axi_bram_ctrl_2
        
        ##Initialise the matrix multiplier IP
        self.matmul = self.overlay.matrix_mult_0
        
    def bram_write(self, bram_id, offset, data):
        if bram_id == 1:
            self.bram1.write(offset, data)
        elif bram_id == 2:
            self.bram2.write(offset, data)
        elif bram_id == 3:
            self.bram3.write(offset, data)
        else:
            print("Can't write! BRAM doesn't exist!")
    
    def bram_read(self, bram_id, offset):
        if bram_id == 1:
            return self.bram1.read(offset)
        elif bram_id == 2:
            return self.bram2.read(offset)
        elif bram_id == 3:
            return self.bram3.read(offset)
        else:
            print(id)
            print("Can't Read! BRAM doesn't exist!")
            return -1
    def matrix_mult(self, a, b):
        ###Write Operation!
        count = 0 
        prod = np.zeros((len(a), len(b[0])), dtype=np.uint32)
        for i in range(16):
            for j in range(16):
                self.bram_write(1, count*4, int(a[i][j]))
                self.bram_write(2, count*4, int(b[i][j]))
                count += 1
        ### Turning AP_START on!
        self.matmul.register_map.CTRL.AP_START = 1
        
        ##Busy Waiting for completion
        while self.matmul.register_map.CTRL.AP_IDLE == 0:
            pass
        
        ##Reading Operation
        count = 0
        for i in range(16):
            for j in range(16):
                prod[i][j] = self.bram_read(3, count*4)
                count += 1
        return prod

In [46]:
inst = matrix_mult_driver('/home/xilinx/jupyter_notebooks/lab2/hls_p4.bit')

In [47]:
import random 
import numpy as np
import time

a = np.ones((16, 16), dtype=np.uint32)
b = np.ones((16, 16), dtype=np.uint32)
prod = np.ones((16, 16), dtype=np.uint32)
est_prod = np.ones((16, 16), dtype=np.uint32)
count = 0
for testcase in range(10):
    ##Building Array A 
    for i in range(16):
        for j in range(16):
            a[i][j] = np.uint32(random.randrange(0, 255, 1))
            b[i][j] = np.uint32(random.randrange(0, 255, 1))
    begin = time.time()
    prod = inst.matrix_mult(a, b)
    end = time.time()
    print("Time: ")
    print(end - begin)
    est_prod = np.matmul(a,b)
    for k in range(len(prod)):
        for l in range(len(prod[0])):
            if prod[k][l] != est_prod[k][l]:
                print("Failed!")
                count += 1
                break
if count > 0:
    print("Cased Failed: ")
    print(count)
else:
    print("PASSED!!")

Time: 
0.04176616668701172
Time: 
0.03268122673034668
Time: 
0.03232240676879883
Time: 
0.03233051300048828
Time: 
0.036614418029785156
Time: 
0.03297996520996094
Time: 
0.03218817710876465
Time: 
0.03255105018615723
Time: 
0.03653717041015625
Time: 
0.032680511474609375
PASSED!!
